### Data file preparation steps:
    
    * The raw text file from https://afltables.com/afl/stats/biglists/bg3.txt is saved as llist.txt
    * The file llist.txt is then processed through the bash script file a_updater.sh to add commas at the appropriate locations and make other modifications, by typing ./a_updater into Terminal - ready to be imported from within Jupyter.

# Preview R16 of 2019 season

In [1]:
#%%html
#<img src="img/ess_stk.png" alt="Drawing" align="left" style="width: 800px;"/>

In [2]:
import os 
os.getcwd() 

'/Users/stevegabriel/data_projects/sports_data/afl_dataexploration'

In [3]:
import pandas as pd

In [4]:
cols = ['seq','date','round','team1','gls_1','bhs_1','pts_1','team2','gls_2','bhs_2','pts_2','venue']

#### The file will contain all matches starting from the first ever VFL game in 1987 - more than fifteen thousand games.

In [5]:
upd_round = pd.read_csv("newlist.csv", names = cols)
trans = upd_round

In [6]:
# Modify index to start at 1 
trans.index += 1
trans.tail()

,seq,date,round,team1,gls_1,bhs_1,pts_1,team2,gls_2,bhs_2,pts_2,venue
15529,15529,29-Jun-2019,R15,Collingwood,5,7,37,North Melbourne,11,15,81,Docklands
15530,15530,29-Jun-2019,R15,Port Adelaide,5,11,41,Western Bulldogs,10,6,66,Adelaide Oval
15531,15531,30-Jun-2019,R15,St Kilda,10,10,70,Richmond,16,7,103,Docklands
15532,15532,30-Jun-2019,R15,Brisbane Lions,15,17,107,Melbourne,11,8,74,Gabba
15533,15533,30-Jun-2019,R15,Fremantle,11,9,75,Carlton,11,13,79,Perth Stadium


#### Shown above, the most four finals matches played on the weekend.

### Adding additional information including state, and addition team names as two letter codes

In [7]:
# Add states
nsw = ['S.C.G.','Sydney Showground','Stadium Australia','Blacktown','Albury']
qld = ['Gabba','Carrara','Cazaly\'s Stadium','Brisbane Exhibition']
sa = ['Football Park','Adelaide Oval']
wa = ['W.A.C.A.','Perth Stadium','Subiaco']
nt = ['Marrara Oval','Traeger Park']
tas = ['York Park','Bellerive Oval','North Hobart']
act = ['Manuka Oval','Bruce Stadium']
intl = ['Wellington','Jiangwan Stadium']

In [8]:
def locations(x):
    if x in nsw:
        return "NSW"
    elif x in sa:
        return "SA"
    elif x in wa:
        return "WA"
    elif x in nt:
        return "NT"
    elif x in tas:
        return "TAS"
    elif x in intl:
        return "INTL"
    elif x in act:
        return "ACT"
    elif x in qld:
        return "QLD"
    else: 
        return "VIC"

In [9]:
trans["state"] = trans["venue"].apply(locations)
trans["state"].value_counts()

VIC     13103
WA        656
SA        587
NSW       524
QLD       487
TAS        95
ACT        50
NT         25
INTL        6
Name: state, dtype: int64

In [10]:
## 1. HVAR & AGGR
trans['hvar'] = trans['pts_1'] - trans['pts_2']
trans['aggr'] = trans['pts_1'] + trans['pts_2']

In [11]:
## 2. YEAR & MONTH
import time
import datetime

In [12]:
trans['date'] = pd.to_datetime(trans['date'])
trans['year'], trans['month'] = trans['date'].dt.year, trans['date'].dt.month

In [13]:
## 3. ABBREVIATED CLUB NAMES
adelaide = ['Adelaide']
brisbane = ['Brisbane Bears','Brisbane Lions']
carlton = ['Carlton']
collingwood = ['Collingwood']
essendon = ['Essendon']
fitzroy = ['Fitzroy']
footscray = ['Footscray']
fremantle = ['Fremantle']
geelong = ['Geelong']
gold_coast = ['Gold Coast']
gws = ['GW Sydney']
hawthorn = ['Hawthorn']
melbourne = ['Melbourne']
north_melbourne = ['North Melbourne','Kangaroos']
port_adelaide = ['Port Adelaide']
richmond = ['Richmond']
south_melbourne = ['South Melbourne']
st_kilda = ['St Kilda']
sydney = ['Sydney']
west_coast = ['West Coast']
western_bulldogs = ['Western Bulldogs', 'Footscray']
university = ['University']

In [14]:
def team_shorten(x):
    if x in adelaide:
        return "AD"
    elif x in brisbane:
        return "BR"
    elif x in carlton:
        return "CA"
    elif x in collingwood:
        return "CO"
    elif x in essendon:
        return "ES"
    elif x in fitzroy:
        return "FI"
    elif x in fremantle:
        return "FR"
    elif x in geelong:
        return "GE"
    elif x in gold_coast:
        return "GC"
    elif x in gws:
        return "GW"
    elif x in hawthorn:
        return "HA"
    elif x in melbourne:
        return "ME"
    elif x in north_melbourne:
        return "NM"
    elif x in port_adelaide:
        return "PA"
    elif x in richmond:
        return "RI"
    elif x in south_melbourne:
        return "SM"
    elif x in st_kilda:
        return "SK"
    elif x in sydney:
        return "SY"
    elif x in west_coast:
        return "WC"
    elif x in western_bulldogs:
        return "WB"
    elif x in university:
        return "UN"
    else: 
        return "NOT_FOUND"

In [15]:
trans["t1"] = trans["team1"].apply(team_shorten)
trans["t2"] = trans["team2"].apply(team_shorten)

In [16]:
trans['t1'].value_counts().sum()

15533

##  FIX TEAM NAMES ANOMOLIES

* Merge Footscray & Western Bulldogs as **Western Bulldogs**
* Merge North Melbourne with Kangaroos as **North Melbourne**
* Merge Brisbane Lions with Brisbane Bears as **Brisbane**
* Leave Sydney and **South Melbourne** as separate
* Leave **Fitzroy** as separate
* Modify GW Sydney to **GWS Giants**

In [17]:
# Function from Analytics Vidhya
def coding(col, codeDict):
  colCoded = pd.Series(col, copy=True)
  for key, value in codeDict.items():
    colCoded.replace(key, value, inplace=True)
  return colCoded

In [18]:
trans["team1"] = coding(trans["team1"], {'Footscray':'Western Bulldogs'})
trans["team2"] = coding(trans["team2"], {'Footscray':'Western Bulldogs'})
trans["team1"] = coding(trans["team1"], {'Kangaroos':'North Melbourne'})
trans["team2"] = coding(trans["team2"], {'Kangaroos':'North Melbourne'})
trans["team1"] = coding(trans["team1"], {'Brisbane Bears':'Brisbane','Brisbane Lions':'Brisbane'})
trans["team2"] = coding(trans["team2"], {'Brisbane Bears':'Brisbane','Brisbane Lions':'Brisbane'})
trans["team1"] = coding(trans["team1"], {'GW Sydney':'GWS Giants'})
trans["team2"] = coding(trans["team2"], {'GW Sydney':'GWS Giants'})

In [19]:
trans["team2"].value_counts()

Collingwood         1281
Carlton             1259
Essendon            1229
Geelong             1226
Melbourne           1198
St Kilda            1196
Richmond            1127
North Melbourne      993
Hawthorn             987
Western Bulldogs     982
Fitzroy              960
South Melbourne      788
Sydney               438
West Coast           383
Brisbane             366
Adelaide             332
Fremantle            280
Port Adelaide        261
Gold Coast            95
GWS Giants            89
University            63
Name: team2, dtype: int64

#### The dataframe with the additional column. Currently it has one game per row.

In [20]:
trans.head(10)

,seq,date,round,team1,gls_1,bhs_1,pts_1,team2,gls_2,bhs_2,pts_2,venue,state,hvar,aggr,year,month,t1,t2
1,1,1897-05-08,R1,Fitzroy,6,13,49,Carlton,2,4,16,Brunswick St,VIC,33,65,1897,5,FI,CA
2,2,1897-05-08,R1,Collingwood,5,11,41,St Kilda,2,4,16,Victoria Park,VIC,25,57,1897,5,CO,SK
3,3,1897-05-08,R1,Geelong,3,6,24,Essendon,7,5,47,Corio Oval,VIC,-23,71,1897,5,GE,ES
4,4,1897-05-08,R1,South Melbourne,3,9,27,Melbourne,6,8,44,Lake Oval,VIC,-17,71,1897,5,SM,ME
5,5,1897-05-15,R2,South Melbourne,6,4,40,Carlton,5,6,36,Lake Oval,VIC,4,76,1897,5,SM,CA
6,6,1897-05-15,R2,Essendon,4,6,30,Collingwood,8,2,50,East Melbourne,VIC,-20,80,1897,5,ES,CO
7,7,1897-05-15,R2,St Kilda,3,8,26,Fitzroy,10,6,66,Junction Oval,VIC,-40,92,1897,5,SK,FI
8,8,1897-05-15,R2,Melbourne,9,10,64,Geelong,3,1,19,M.C.G.,VIC,45,83,1897,5,ME,GE
9,9,1897-05-22,R3,Collingwood,6,5,41,Geelong,5,7,37,Victoria Park,VIC,4,78,1897,5,CO,GE
10,10,1897-05-22,R3,Fitzroy,5,9,39,Melbourne,7,8,50,Brunswick St,VIC,-11,89,1897,5,FI,ME


## ALL DRAWS (BOOLEAN)

In [21]:
def reg_draw(hvar):
    if hvar == 0:
        return 1
    else: return 0

In [22]:
trans["draw_rt"] = trans["hvar"].apply(reg_draw)

In [23]:
trans['draw_rt'].value_counts()

0    15375
1      158
Name: draw_rt, dtype: int64

### The 158 draws above, represent the VFL/AFL games that have concluded in a draw result

### The two ET (extra-time) draws below are statistically relevent, since they also ended regular full-time with both teams on the same score. However extra-time was applied to create a result, due to finals constraints.

In [24]:
def et_draw(seq):
    if seq == 10794:
        return 1
    elif seq == 13203:
        return 1
    else: return 0

In [25]:
trans["etime_draw"] = trans["seq"].apply(et_draw)
trans['etime_draw'].value_counts()

0    15531
1        2
Name: etime_draw, dtype: int64

In [26]:
### Add the two columns
trans['draws_all']  = trans['draw_rt'] + trans['etime_draw']
trans['draws_all'].value_counts()

0    15373
1      160
Name: draws_all, dtype: int64

In [27]:
# Check
trans.iloc[:,:][(trans.etime_draw == 1)]

,seq,date,round,team1,gls_1,bhs_1,pts_1,team2,gls_2,bhs_2,...,state,hvar,aggr,year,month,t1,t2,draw_rt,etime_draw,draws_all
10794,10794,1994-09-10,QF,North Melbourne,15,24,114,Hawthorn,13,13,...,VIC,23,205,1994,9,NM,HA,0,1,1
13203,13203,2007-09-14,SF,Collingwood,13,15,93,West Coast,10,14,...,WA,19,167,2007,9,CO,WC,0,1,1


In [28]:
# 6. GAME CATEGORY
finals = ['EF','QF','SF','PF','GF']

In [29]:
def game_cat(x):
    if x in finals:
        return "final"
    else: 
        return "reg_season"

In [30]:
trans["type"] = trans["round"].apply(game_cat)

In [31]:
trans['type'].value_counts()

reg_season    14878
final           655
Name: type, dtype: int64

### So, there have been 650 finals up to the end of week one of 2018 finals series.

In [32]:
def grouping_2K(seq):
    n=500
    if seq <= 4*n:
        return 1
    elif seq <= 8*n:
        return 2
    elif seq <= 12*n:
        return 3
    elif seq <= 16*n:
        return 4
    elif seq <= 20*n:
        return 5
    elif seq <= 24*n:
        return 6
    elif seq <= 28*n:
        return 7
    else: return 8

In [33]:
trans["grp_2K"] = trans["seq"].apply(grouping_2K)

In [34]:
def grouping_1K(seq):
    n=1000
    if seq <= n:
        return 1
    elif seq <= 2*n:
        return 2
    elif seq <= 3*n:
        return 3
    elif seq <= 4*n:
        return 4
    elif seq <= 5*n:
        return 5
    elif seq <= 6*n:
        return 6
    elif seq <= 7*n:
        return 7
    elif seq <= 8*n:
        return 8
    elif seq <= 9*n:
        return 9
    elif seq <= 10*n:
        return 10
    elif seq <= 11*n:
        return 11
    elif seq <= 12*n:
        return 12
    elif seq <= 13*n:
        return 13
    elif seq <= 14*n:
        return 14
    elif seq <= 15*n:
        return 15
    else: return 16

In [35]:
trans["grp_1K"] = trans["seq"].apply(grouping_1K)

In [36]:
# Export the data frame for use in afl draws EDA
trans.to_csv('all_afl_wide.csv')

## ADD res1 and res2 columns as categorical 1 = win, 2 = loss, 3 = draw based on hvar (ignoring the two extra time finals)

In [37]:
def result_cat_h(hvar):
    if hvar > 0:
        return 1
    elif hvar < 0:
        return 2
    else: return 3

In [38]:
def result_cat_a(hvar):
    if hvar > 0:
        return 2
    elif hvar < 0:
        return 1
    else: return 3

In [39]:
trans['res1'] = trans['hvar'].apply(result_cat_h)
trans['res2'] = trans['hvar'].apply(result_cat_a)

In [40]:
trans.tail(9)

,seq,date,round,team1,gls_1,bhs_1,pts_1,team2,gls_2,bhs_2,...,t1,t2,draw_rt,etime_draw,draws_all,type,grp_2K,grp_1K,res1,res2
15525,15525,2019-06-27,R15,Essendon,12,5,77,GWS Giants,10,11,...,ES,GW,0,0,0,reg_season,8,16,1,2
15526,15526,2019-06-28,R15,Geelong,14,12,96,Adelaide,10,9,...,GE,AD,0,0,0,reg_season,8,16,1,2
15527,15527,2019-06-29,R15,Hawthorn,9,17,71,West Coast,11,11,...,HA,WC,0,0,0,reg_season,8,16,2,1
15528,15528,2019-06-29,R15,Sydney,14,9,93,Gold Coast,7,9,...,SY,GC,0,0,0,reg_season,8,16,1,2
15529,15529,2019-06-29,R15,Collingwood,5,7,37,North Melbourne,11,15,...,CO,NM,0,0,0,reg_season,8,16,2,1
15530,15530,2019-06-29,R15,Port Adelaide,5,11,41,Western Bulldogs,10,6,...,PA,WB,0,0,0,reg_season,8,16,2,1
15531,15531,2019-06-30,R15,St Kilda,10,10,70,Richmond,16,7,...,SK,RI,0,0,0,reg_season,8,16,2,1
15532,15532,2019-06-30,R15,Brisbane,15,17,107,Melbourne,11,8,...,BR,ME,0,0,0,reg_season,8,16,1,2
15533,15533,2019-06-30,R15,Fremantle,11,9,75,Carlton,11,13,...,FR,CA,0,0,0,reg_season,8,16,2,1


## STAGE TWO
## 9. Reduce and re-order the data frame to prepare for for Long format

In [41]:
tracker = trans
tracker = tracker[['year', 'team1','pts_1','team2','pts_2','venue','t1','t2','hvar']]
tracker.tail()

,year,team1,pts_1,team2,pts_2,venue,t1,t2,hvar
15529,2019,Collingwood,37,North Melbourne,81,Docklands,CO,NM,-44
15530,2019,Port Adelaide,41,Western Bulldogs,66,Adelaide Oval,PA,WB,-25
15531,2019,St Kilda,70,Richmond,103,Docklands,SK,RI,-33
15532,2019,Brisbane,107,Melbourne,74,Gabba,BR,ME,33
15533,2019,Fremantle,75,Carlton,79,Perth Stadium,FR,CA,-4


### Create two new columns with values 1 and 0 for the two subsets

In [42]:
tracker['subset_1'] = 1
tracker['subset_2'] = 0

/Users/stevegabriel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/stevegabriel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [43]:
tracker.tail(9)

,year,team1,pts_1,team2,pts_2,venue,t1,t2,hvar,subset_1,subset_2
15525,2019,Essendon,77,GWS Giants,71,Docklands,ES,GW,6,1,0
15526,2019,Geelong,96,Adelaide,69,Kardinia Park,GE,AD,27,1,0
15527,2019,Hawthorn,71,West Coast,77,M.C.G.,HA,WC,-6,1,0
15528,2019,Sydney,93,Gold Coast,51,S.C.G.,SY,GC,42,1,0
15529,2019,Collingwood,37,North Melbourne,81,Docklands,CO,NM,-44,1,0
15530,2019,Port Adelaide,41,Western Bulldogs,66,Adelaide Oval,PA,WB,-25,1,0
15531,2019,St Kilda,70,Richmond,103,Docklands,SK,RI,-33,1,0
15532,2019,Brisbane,107,Melbourne,74,Gabba,BR,ME,33,1,0
15533,2019,Fremantle,75,Carlton,79,Perth Stadium,FR,CA,-4,1,0


In [44]:
droplist_1 = ['subset_2']
droplist_2 = ['subset_1']

In [45]:
left_team = tracker.drop(droplist_1, axis = 1)
right_team = tracker.drop(droplist_2, axis = 1)

In [46]:
left_team.shape

(15533, 10)

In [47]:
right_team.shape

(15533, 10)

# Rename columns in each dframe 

In [48]:
left_team.columns = ['year','team','tm_pts','opp_team','op_pts','venue','tm','op','hvar','lr']
right_team.columns = ['year','opp_team','op_pts','team','tm_pts','venue','op','tm','hvar','lr']

### Now re-order the second dframe to match columns

### Note double brackets to re-order

In [49]:
right_team = right_team[['year', 'team','tm_pts','opp_team','op_pts','venue','tm','op','hvar','lr']]
right_team.tail()

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr
15529,2019,North Melbourne,81,Collingwood,37,Docklands,NM,CO,-44,0
15530,2019,Western Bulldogs,66,Port Adelaide,41,Adelaide Oval,WB,PA,-25,0
15531,2019,Richmond,103,St Kilda,70,Docklands,RI,SK,-33,0
15532,2019,Melbourne,74,Brisbane,107,Gabba,ME,BR,33,0
15533,2019,Carlton,79,Fremantle,75,Perth Stadium,CA,FR,-4,0


In [50]:
right_team.tail()

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr
15529,2019,North Melbourne,81,Collingwood,37,Docklands,NM,CO,-44,0
15530,2019,Western Bulldogs,66,Port Adelaide,41,Adelaide Oval,WB,PA,-25,0
15531,2019,Richmond,103,St Kilda,70,Docklands,RI,SK,-33,0
15532,2019,Melbourne,74,Brisbane,107,Gabba,ME,BR,33,0
15533,2019,Carlton,79,Fremantle,75,Perth Stadium,CA,FR,-4,0


In [51]:
left_team.tail()

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr
15529,2019,Collingwood,37,North Melbourne,81,Docklands,CO,NM,-44,1
15530,2019,Port Adelaide,41,Western Bulldogs,66,Adelaide Oval,PA,WB,-25,1
15531,2019,St Kilda,70,Richmond,103,Docklands,SK,RI,-33,1
15532,2019,Brisbane,107,Melbourne,74,Gabba,BR,ME,33,1
15533,2019,Fremantle,75,Carlton,79,Perth Stadium,FR,CA,-4,1


## CONCATENATION

### Can now concatenate the two dframes, as in si_week11

In [52]:
merge_ft = pd.concat([left_team, right_team], axis=0)
merge_ft.head()

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr
1,1897,Fitzroy,49,Carlton,16,Brunswick St,FI,CA,33,1
2,1897,Collingwood,41,St Kilda,16,Victoria Park,CO,SK,25,1
3,1897,Geelong,24,Essendon,47,Corio Oval,GE,ES,-23,1
4,1897,South Melbourne,27,Melbourne,44,Lake Oval,SM,ME,-17,1
5,1897,South Melbourne,40,Carlton,36,Lake Oval,SM,CA,4,1


#### The new long data frame consisting of 30,000 plus rows:

In [53]:
merge_ft.shape

(31066, 10)

# The following is to keep each game together (two rows per game)

In [54]:
merge_ft.sort_index(inplace=True)
merge_ft.head(10)

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr
1,1897,Fitzroy,49,Carlton,16,Brunswick St,FI,CA,33,1
1,1897,Carlton,16,Fitzroy,49,Brunswick St,CA,FI,33,0
2,1897,Collingwood,41,St Kilda,16,Victoria Park,CO,SK,25,1
2,1897,St Kilda,16,Collingwood,41,Victoria Park,SK,CO,25,0
3,1897,Geelong,24,Essendon,47,Corio Oval,GE,ES,-23,1
3,1897,Essendon,47,Geelong,24,Corio Oval,ES,GE,-23,0
4,1897,South Melbourne,27,Melbourne,44,Lake Oval,SM,ME,-17,1
4,1897,Melbourne,44,South Melbourne,27,Lake Oval,ME,SM,-17,0
5,1897,South Melbourne,40,Carlton,36,Lake Oval,SM,CA,4,1
5,1897,Carlton,36,South Melbourne,40,Lake Oval,CA,SM,4,0


In [55]:
merge_ft.tail()

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr
15531,2019,St Kilda,70,Richmond,103,Docklands,SK,RI,-33,1
15532,2019,Melbourne,74,Brisbane,107,Gabba,ME,BR,33,0
15532,2019,Brisbane,107,Melbourne,74,Gabba,BR,ME,33,1
15533,2019,Fremantle,75,Carlton,79,Perth Stadium,FR,CA,-4,1
15533,2019,Carlton,79,Fremantle,75,Perth Stadium,CA,FR,-4,0


## There are actually 30,804 rows, indexing isn't  sequential

## Create team result column

In [56]:
def neg_hvar(hvar):
        return -1*hvar

In [57]:
merge_ft["neg_dummy"] = merge_ft['hvar'].apply(neg_hvar)

In [58]:
merge_ft.tail()

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr,neg_dummy
15531,2019,St Kilda,70,Richmond,103,Docklands,SK,RI,-33,1,33
15532,2019,Melbourne,74,Brisbane,107,Gabba,ME,BR,33,0,-33
15532,2019,Brisbane,107,Melbourne,74,Gabba,BR,ME,33,1,-33
15533,2019,Fremantle,75,Carlton,79,Perth Stadium,FR,CA,-4,1,4
15533,2019,Carlton,79,Fremantle,75,Perth Stadium,CA,FR,-4,0,4


In [59]:
def team_var(hvar):
    if merge_ft['lr'] == 1:
        return hvar
    else: return neg_dummy

In [60]:
def new_lr(lr):
    if lr == 0:
        return -1
    else: return 1

In [61]:
merge_ft["new_lr"] = merge_ft['lr'].apply(new_lr)
merge_ft["tm_var"] = merge_ft['new_lr']*merge_ft['hvar']

In [62]:
merge_ft.tail()

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr,neg_dummy,new_lr,tm_var
15531,2019,St Kilda,70,Richmond,103,Docklands,SK,RI,-33,1,33,1,-33
15532,2019,Melbourne,74,Brisbane,107,Gabba,ME,BR,33,0,-33,-1,-33
15532,2019,Brisbane,107,Melbourne,74,Gabba,BR,ME,33,1,-33,1,33
15533,2019,Fremantle,75,Carlton,79,Perth Stadium,FR,CA,-4,1,4,1,-4
15533,2019,Carlton,79,Fremantle,75,Perth Stadium,CA,FR,-4,0,4,-1,4


### Can now remove the columns new_lr and neg_dummy, which were used to create tm_var

In [63]:
merge_ft = merge_ft.drop(['neg_dummy', 'new_lr'], axis=1)

In [64]:
merge_ft.head()

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr,tm_var
1,1897,Fitzroy,49,Carlton,16,Brunswick St,FI,CA,33,1,33
1,1897,Carlton,16,Fitzroy,49,Brunswick St,CA,FI,33,0,-33
2,1897,Collingwood,41,St Kilda,16,Victoria Park,CO,SK,25,1,25
2,1897,St Kilda,16,Collingwood,41,Victoria Park,SK,CO,25,0,-25
3,1897,Geelong,24,Essendon,47,Corio Oval,GE,ES,-23,1,-23


### Note that the column 'lr' denotes left right, so there are only two different values (1 denotes left)

In [65]:
merge_ft['lr'].value_counts()

1    15533
0    15533
Name: lr, dtype: int64

## New function for win/loss/draw (W/L/D)


In [66]:
def outcome(tm_var):
    if tm_var >= 1:
        return "W"
    elif tm_var <= -1:
        return "L"
    elif tm_var == 0:
        return "D"
    else: return "Unknown"

In [67]:
merge_ft["outcome"] = merge_ft['tm_var'].apply(outcome)

In [68]:
merge_ft.head()

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr,tm_var,outcome
1,1897,Fitzroy,49,Carlton,16,Brunswick St,FI,CA,33,1,33,W
1,1897,Carlton,16,Fitzroy,49,Brunswick St,CA,FI,33,0,-33,L
2,1897,Collingwood,41,St Kilda,16,Victoria Park,CO,SK,25,1,25,W
2,1897,St Kilda,16,Collingwood,41,Victoria Park,SK,CO,25,0,-25,L
3,1897,Geelong,24,Essendon,47,Corio Oval,GE,ES,-23,1,-23,L


In [69]:
merge_ft.tail(9)

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr,tm_var,outcome
15529,2019,Collingwood,37,North Melbourne,81,Docklands,CO,NM,-44,1,-44,L
15530,2019,Western Bulldogs,66,Port Adelaide,41,Adelaide Oval,WB,PA,-25,0,25,W
15530,2019,Port Adelaide,41,Western Bulldogs,66,Adelaide Oval,PA,WB,-25,1,-25,L
15531,2019,Richmond,103,St Kilda,70,Docklands,RI,SK,-33,0,33,W
15531,2019,St Kilda,70,Richmond,103,Docklands,SK,RI,-33,1,-33,L
15532,2019,Melbourne,74,Brisbane,107,Gabba,ME,BR,33,0,-33,L
15532,2019,Brisbane,107,Melbourne,74,Gabba,BR,ME,33,1,33,W
15533,2019,Fremantle,75,Carlton,79,Perth Stadium,FR,CA,-4,1,-4,L
15533,2019,Carlton,79,Fremantle,75,Perth Stadium,CA,FR,-4,0,4,W


## Research for Week 16 of 2019 using Pandas filtering:
### * Essendon v GWS
### * Hawthorn v WC

### Recent contests NTH v STK

In [75]:
merge_ft.iloc[:,:][(merge_ft.tm == 'SK') & (merge_ft.op == 'NM')].tail(10)

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr,tm_var,outcome
14076,2012,St Kilda,103,North Melbourne,136,Docklands,SK,NM,-33,1,-33,L
14255,2013,St Kilda,65,North Melbourne,133,Docklands,SK,NM,68,0,-68,L
14511,2014,St Kilda,33,North Melbourne,92,Bellerive Oval,SK,NM,59,0,-59,L
14744,2015,St Kilda,83,North Melbourne,120,Bellerive Oval,SK,NM,37,0,-37,L
14848,2016,St Kilda,75,North Melbourne,82,Docklands,SK,NM,-7,1,-7,L
14944,2016,St Kilda,62,North Melbourne,85,Docklands,SK,NM,23,0,-23,L
15097,2017,St Kilda,89,North Melbourne,72,Docklands,SK,NM,-17,0,17,W
15181,2017,St Kilda,127,North Melbourne,78,Docklands,SK,NM,49,1,49,W
15211,2018,St Kilda,43,North Melbourne,95,Docklands,SK,NM,52,0,-52,L
15398,2018,St Kilda,94,North Melbourne,117,Docklands,SK,NM,-23,1,-23,L


### Recent contests MEv CA

In [76]:
merge_ft.iloc[:,:][(merge_ft.tm == 'CA') & (merge_ft.op == 'ME') & (merge_ft.venue != 'Subiaco')].tail(12)

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr,tm_var,outcome
13554,2009,Carlton,153,Melbourne,96,Docklands,CA,ME,57,1,57,W
13659,2010,Carlton,101,Melbourne,60,M.C.G.,CA,ME,41,1,41,W
13833,2011,Carlton,93,Melbourne,46,M.C.G.,CA,ME,-47,0,47,W
13913,2011,Carlton,134,Melbourne,58,M.C.G.,CA,ME,76,1,76,W
14039,2012,Carlton,107,Melbourne,49,M.C.G.,CA,ME,58,1,58,W
14219,2013,Carlton,121,Melbourne,60,M.C.G.,CA,ME,61,1,61,W
14402,2014,Carlton,58,Melbourne,81,M.C.G.,CA,ME,-23,1,-23,L
14758,2015,Carlton,78,Melbourne,55,M.C.G.,CA,ME,23,1,23,W
14973,2016,Carlton,78,Melbourne,58,M.C.G.,CA,ME,20,1,20,W
15010,2017,Carlton,64,Melbourne,86,M.C.G.,CA,ME,22,0,-22,L
